# General DA fittings and Fokker-Planck comparisons


## Setup scripts if we are under SWAN

In [ ]:
# Working in the right path
%cd /eos/project/d/da-and-diffusion-studies/DA_Studies/Simulations/Models/loss_studies/notebooks

In [ ]:
# Install the libraries
import sys
!{sys.executable} -m pip install --user tqdm pynverse sixtrackwrap crank-nicolson-numba henon-map symplectic-map
!{sys.executable} -m pip install --user --upgrade sixtrackwrap 
!{sys.executable} -m pip install --user --upgrade crank-nicolson-numba 
!{sys.executable} -m pip install --user --upgrade henon-map 
!{sys.executable} -m pip install --user --upgrade symplectic-map
!export PYTHONPATH=$CERNBOX_HOME.local/lib/python3.7/site-packages:$PYTHONPATH

In [ ]:
# For this "presentation" only!
import warnings
warnings.filterwarnings('ignore')

## Imports

In [3]:
%matplotlib widget

In [4]:
# Base libraries
import math
import numpy as np
import scipy.integrate as integrate
from scipy.special import erf
import pickle
import itertools
from scipy.optimize import curve_fit

from numba import njit, prange

from tqdm.notebook import tqdm
import time
import matplotlib.pyplot as plt
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.ticker as ticker
from math import gcd

import pandas as pd

from scipy.special import lambertw
from scipy.interpolate import interp1d

import os

# Personal libraries
import sixtrackwrap as sx
import crank_nicolson_numba.nekhoroshev as nk

# Personal modules
import fit_utils as fit

## Load data and setup original DA

In [3]:
savepath = "../data/"

engine = sx.uniform_radial_scanner.load_values(savepath + "big_scan.pkl")

min_turns = engine.min_time
max_turns = engine.max_time
n_turn_samples = 500

turn_sampling = np.linspace(min_turns, max_turns, n_turn_samples, dtype=np.int_)[::-1]

d_r = engine.dr
starting_step = engine.starting_step

# BASELINE COMPUTING
baseline_samples = 33
baseline_total_samples = baseline_samples ** 3

In [ ]:
DA = engine.compute_DA_standard(turn_sampling)

In [7]:
axis_sampling = np.concatenate((turn_sampling, [0.0]))

## Visualize original DA value from data

In [13]:
fig1, ax1 = plt.subplots()
ax1.plot(turn_sampling, DA)
ax1.set_title("Original $DA$ values (LHC on SixTrack)")
ax1.set_xlabel("$N$ turns")
ax1.set_ylabel("$DA(N)$")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, '$DA(N)$')

## Setup Dataframe for storing the fitting data properly!

In [14]:
labels = (
    ("type", ""),
    ("sigma", ""),
    ("Model 2", "k"),
    ("Model 2", "k err"),
    ("Model 2", "rho"),
    ("Model 2", "rho err"),
    ("Model 2", "N0"),
    ("Model 2", "N0 err"),
    ("Model 2", "Chi2"),
    ("Model 4 (2 free pars)", "k"),
    ("Model 4 (2 free pars)", "k err"),
    ("Model 4 (2 free pars)", "rho"),
    ("Model 4 (2 free pars)", "rho err"),
    ("Model 4 (2 free pars)", "lambda"),
    ("Model 4 (2 free pars)", "Chi2"),
    ("Model 4 (3 free pars)", "k"),
    ("Model 4 (3 free pars)", "k err"),
    ("Model 4 (3 free pars)", "rho"),
    ("Model 4 (3 free pars)", "rho err"),
    ("Model 4 (3 free pars)", "N0"),
    ("Model 4 (3 free pars)", "N0 err"),
    ("Model 4 (3 free pars)", "lambda"),
    ("Model 4 (3 free pars)", "Chi2"),
)

fitting_data = pd.DataFrame(columns=pd.MultiIndex.from_tuples(labels))

## How is the error on the DA loss computed right now?

1. Consider all the radiuses sampled.
2. Compute the DA value.
3. For every radius sampled, compute the difference from the DA value.
4. The absolute value of the average of all these differences is considered as error.

(I tried using the Standard Deviation of the radiuses distribution, but it ended up being 10% of the DA itself, so we "need" somehow a smaller error estimation)

## Loss and Fits -- Symmetric Gaussian Beam Distribution
### Choose your Sigma and DA_0!

In [29]:
sigma = 8.0
DA0 = 36.0

### Loss comparison

In [30]:
# Assign uniform weights to engine and compute loss
engine.assign_weights(
    sx.assign_symmetric_gaussian(sigma)
)
real_values = engine.compute_loss(turn_sampling, DA0)
real_values = np.concatenate((real_values, [1.0]))

# Compute DA-based loss
values = fit.symmetric_gaussian_loss(DA, sigma, DA0)
values = np.concatenate((values, [1.0]))

#### Visualize Loss comparison

In [31]:
fig2, ax2 = plt.subplots()
ax2.plot(axis_sampling, values, label="Values from DA")
ax2.plot(axis_sampling, real_values, label="Values from weights")
ax2.legend()
ax2.set_xlabel("$N$ turns")
ax2.set_ylabel("Active beam")
ax2.set_title("Symmetric Gaussian beam ($\\sigma={}$, Cutting Point at $DA=26.0$)".format(sigma))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Symmetric Gaussian beam ($\\sigma=8.0$, Cutting Point at $DA=26.0$)')

### Get DA from loss

In [32]:
DA_from_loss = fit.DA_from_symmetric_gaussian_loss(real_values, sigma, DA0)

#### Visualize difference

In [33]:
fig3, ax3 = plt.subplots()
ax3.plot(turn_sampling, DA, label="Original DA")
ax3.plot(axis_sampling[:-1], DA_from_loss[:-1], label="DA extracted from real loss values")
ax3.legend()
ax3.set_xlabel("$N$ turns")
ax3.set_ylabel("$DA(N)$")
ax3.set_title("Comparison between original DA and DA obtained from real Loss")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Comparison between original DA and DA obtained from real Loss')

### Compare fitting values with Model 2

In [34]:
k_min = 0.05
k_max = 0.20
samples = 400

ks = np.linspace(k_min, k_max, samples)

real_pars, real_errs, real_co_pars = fit.explore_k_model_2(turn_sampling, DA, k_min, k_max, samples)
loss_pars, loss_errs, loss_co_pars = fit.explore_k_model_2(axis_sampling[:-1], DA_from_loss[:-1], k_min, k_max, samples)

real_selected_err_2 = np.min(real_errs)
real_selected_k_2 = ks[np.argmin(real_errs)]
real_selected_pars_2 = real_pars[np.argmin(real_errs)]
real_selected_co_pars_2 = real_co_pars[np.argmin(real_errs)]

loss_selected_err_2 = np.min(loss_errs)
loss_selected_k_2 = ks[np.argmin(loss_errs)]
loss_selected_pars_2 = loss_pars[np.argmin(loss_errs)]
loss_selected_co_pars_2 = loss_co_pars[np.argmin(loss_errs)]

In [35]:
print("REAL:", "$\\kappa:$", real_selected_k_2, ", $\\rho_\\ast$ and $N_0$", real_selected_pars_2)
print("FROM LOSS:", "$\\kappa:$", loss_selected_k_2, ", $\\rho_\\ast$ and $N_0$", loss_selected_pars_2)

REAL: $\kappa:$ 0.11992481203007521 , $\rho_\ast$ and $N_0$ [36.15442002 46.76235005]
FROM LOSS: $\kappa:$ 0.11729323308270678 , $\rho_\ast$ and $N_0$ [36.03135337 37.12036537]


In [36]:
fig3, ax3 = plt.subplots()

ax3.plot(turn_sampling, DA, label="Original DA")
ax3.plot(turn_sampling, fit.model_2(turn_sampling, real_selected_pars_2[0], real_selected_pars_2[1], real_selected_k_2), label="Fitting on original DA\n$\\kappa = {:.4f},\\rho_\\ast = {:.4f}, N_0={:.4f}$".format(real_selected_k_2, real_selected_pars_2[0], real_selected_pars_2[1] ))

ax3.plot(axis_sampling[:-1], DA_from_loss[:-1], label="DA extracted from real loss values")
ax3.plot(turn_sampling, fit.model_2(turn_sampling, loss_selected_pars_2[0], loss_selected_pars_2[1], loss_selected_k_2), label="Fitting on original DA\n$\\kappa = {:.4f},\\rho_\\ast = {:.4f}, N_0={:.4f}$".format(loss_selected_k_2, loss_selected_pars_2[0], loss_selected_pars_2[1] ))

ax3.legend()
ax3.set_xlabel("$N$ turns")
ax3.set_ylabel("$DA(N)$")
ax3.set_title("Comparison between original DA and DA obtained from real Loss, with Model 2 Fits")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Comparison between original DA and DA obtained from real Loss, with Model 2 Fits')

### Compare fitting values with Model 4

In [37]:
k_min = 0.15
k_max = 0.30
samples = 400

ks = np.linspace(k_min, k_max, samples)

real_pars, real_errs, real_co_pars = fit.explore_k_model_4(turn_sampling, DA, k_min, k_max, samples)
loss_pars, loss_errs, loss_co_pars = fit.explore_k_model_4(axis_sampling[:-1], DA_from_loss[:-1], k_min, k_max, samples)

real_selected_err_4 = np.min(real_errs)
real_selected_k_4 = ks[np.argmin(real_errs)]
real_selected_pars_4 = real_pars[np.argmin(real_errs)]
real_selected_co_pars_4 = real_co_pars[np.argmin(real_errs)]

loss_selected_err_4 = np.min(loss_errs)
loss_selected_k_4 = ks[np.argmin(loss_errs)]
loss_selected_pars_4 = loss_pars[np.argmin(loss_errs)]
loss_selected_co_pars_4 = loss_co_pars[np.argmin(loss_errs)]

In [38]:
print("REAL:", "$\\kappa:$", real_selected_k_4, ", $\\rho_\\ast$ and $N_0$", real_selected_pars_4)
print("FROM LOSS:", "$\\kappa:$", loss_selected_k_4, ", $\\rho_\\ast$ and $N_0$", loss_selected_pars_4)

REAL: $\kappa:$ 0.21616541353383456 , $\rho_\ast$ and $N_0$ [60.22905065]
FROM LOSS: $\kappa:$ 0.21729323308270676 , $\rho_\ast$ and $N_0$ [60.76538463]


In [39]:
fig3, ax3 = plt.subplots()

ax3.plot(turn_sampling, DA, label="Original DA")
ax3.plot(turn_sampling, fit.model_4(turn_sampling, real_selected_pars_4[0], real_selected_k_4), label="Fitting on original DA\n$\\kappa = {:.4f},\\rho_\\ast = {:.4f}$".format(real_selected_k_4, real_selected_pars_4[0]))

ax3.plot(axis_sampling[:-1], DA_from_loss[:-1], label="DA extracted from real loss values")
ax3.plot(turn_sampling, fit.model_4(turn_sampling, loss_selected_pars_4[0], loss_selected_k_4), label="Fitting on original DA\n$\\kappa = {:.4f},\\rho_\\ast = {:.4f}$".format(loss_selected_k_4, loss_selected_pars_4[0]))

ax3.legend()
ax3.set_xlabel("$N$ turns")
ax3.set_ylabel("$DA(N)$")
ax3.set_title("Comparison between original DA and DA obtained from real Loss, with Model 4 Fits $(\\lambda=0.5)$")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Comparison between original DA and DA obtained from real Loss, with Model 4 Fits $(\\lambda=0.5)$')

### Compare fitting values with Model 4 (But more free)

In [40]:
k_min = 0.05
k_max = 0.20
samples = 400

ks = np.linspace(k_min, k_max, samples)

real_pars, real_errs, real_co_pars = fit.explore_model_4_free(turn_sampling, DA, k_min, k_max, samples)
loss_pars, loss_errs, loss_co_pars = fit.explore_model_4_free(axis_sampling[:-1], DA_from_loss[:-1], k_min, k_max, samples)

real_selected_err_4_free = np.min(real_errs)
real_selected_k_4_free = ks[np.argmin(real_errs)]
real_selected_pars_4_free = real_pars[np.argmin(real_errs)]
real_selected_co_pars_4_free = real_co_pars[np.argmin(real_errs)]

loss_selected_err_4_free = np.min(loss_errs)
loss_selected_k_4_free = ks[np.argmin(loss_errs)]
loss_selected_pars_4_free = loss_pars[np.argmin(loss_errs)]
loss_selected_co_pars_4_free = loss_co_pars[np.argmin(loss_errs)]

In [41]:
print("REAL:", "$\\kappa:$", real_selected_k_4_free, ", $\\rho_\\ast$ and $N_0$", real_selected_pars_4_free)
print("FROM LOSS:", "$\\kappa:$", loss_selected_k_4_free, ", $\\rho_\\ast$ and $N_0$", loss_selected_pars_4_free)

REAL: $\kappa:$ 0.12293233082706768 , $\rho_\ast$ and $N_0$ [36.79845342 43.27224119]
FROM LOSS: $\kappa:$ 0.11992481203007521 , $\rho_\ast$ and $N_0$ [36.59997887 34.69222847]


In [42]:
fig3, ax3 = plt.subplots()

ax3.plot(turn_sampling, DA, label="Original DA")
ax3.plot(turn_sampling, fit.model_4_free(turn_sampling, real_selected_pars_4_free[0], real_selected_pars_4_free[1], real_selected_k_4_free), label="Fitting on original DA\n$\\kappa = {:.4f},\\rho_\\ast = {:.4f}, N_0 = {:.4f}$".format(real_selected_k_4_free, real_selected_pars_4_free[0], real_selected_pars_4_free[1]))

ax3.plot(axis_sampling[:-1], DA_from_loss[:-1], label="DA extracted from real loss values")
ax3.plot(turn_sampling, fit.model_4_free(turn_sampling, loss_selected_pars_4_free[0], loss_selected_pars_4_free[1], loss_selected_k_4_free), label="Fitting on original DA\n$\\kappa = {:.4f},\\rho_\\ast = {:.4f}, N_0 = {:.4f}$".format(loss_selected_k_4_free, loss_selected_pars_4_free[0], loss_selected_pars_4_free[1]))

ax3.legend()
ax3.set_xlabel("$N$ turns")
ax3.set_ylabel("$DA(N)$")
ax3.set_title("Comparison between original DA and DA obtained from real Loss,\n with Model 4 Fits $(\\lambda=0.5)$, 3 free parameters")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Comparison between original DA and DA obtained from real Loss,\n with Model 4 Fits $(\\lambda=0.5)$, 3 free parameters')

### Save data in the dataframe

In [ ]:
fitting_data.loc[len(fitting_data)] = [
     "real",
     np.nan,
     real_selected_k_2,
     ks[1] - ks[0],
     real_selected_pars_2[0],
     np.sqrt(real_selected_co_pars_2[0, 0]), 
     real_selected_pars_2[1], 
     np.sqrt(real_selected_co_pars_2[1, 1]), 
     real_selected_err_2,
     real_selected_k_4, 
     ks[1] - ks[0], 
     real_selected_pars_4[0], 
     np.sqrt(real_selected_co_pars_4[0, 0]),
     1/2,
     real_selected_err_4,
     real_selected_k_4_free, 
     ks[1] - ks[0], 
     real_selected_pars_4_free[0], 
     np.sqrt(real_selected_co_pars_4_free[0, 0]),
     real_selected_pars_4_free[1], 
     np.sqrt(real_selected_co_pars_4_free[1, 1]),
     1/2,
     real_selected_err_4_free
]

In [43]:
fitting_data.loc[len(fitting_data)] = [
    "gaussian",
     sigma,
     loss_selected_k_2,
     ks[1] - ks[0],
     loss_selected_pars_2[0],
     np.sqrt(loss_selected_co_pars_2[0, 0]),
     loss_selected_pars_2[1],
     np.sqrt(loss_selected_co_pars_2[1, 1]),
     loss_selected_err_2,
     loss_selected_k_4,
     ks[1] - ks[0],
     loss_selected_pars_4[0],
     np.sqrt(loss_selected_co_pars_4[0, 0]),
     1/2,
     loss_selected_err_4,
     loss_selected_k_4_free,
     ks[1] - ks[0],
     loss_selected_pars_4_free[1],
     np.sqrt(loss_selected_co_pars_4_free[1, 1]),
     loss_selected_pars_4_free[0],
     np.sqrt(loss_selected_co_pars_4_free[0, 0]),
     1/2,
     loss_selected_err_4_free
]

## Display general dataframe with fitting data

In [44]:
fitting_data[["type", "sigma", "Model 2"]]

type sigma   Model 2                                            \
                          k     k err        rho   rho err         N0   
0      real   NaN  0.119925  0.000376  36.154420  0.003738  46.762350   
1   uniform   NaN  0.121429  0.000376  37.482097  0.009520  57.090429   
2  gaussian   8.0  0.117293  0.000376  36.031353  0.010386  37.120365   

                        
     N0 err       Chi2  
0  0.203332   0.008032  
1  0.543469  10.872958  
2  0.498382   0.208608

In [45]:
fitting_data[["type", "sigma", "Model 4 (2 free pars)"]]

type sigma Model 4 (2 free pars)                                        \
                                      k     k err        rho   rho err lambda   
0      real   NaN              0.216165  0.000376  60.229051  0.012648    0.5   
1   uniform   NaN              0.223684  0.000376  64.376111  0.020638    0.5   
2  gaussian   8.0              0.217293  0.000376  60.765385  0.015983    0.5   

              
        Chi2  
0   0.180550  
1  11.032593  
2   0.323162

In [46]:
fitting_data[["type", "sigma", "Model 4 (3 free pars)"]]

type sigma Model 4 (3 free pars)                                 \
                                      k     k err        rho   rho err   
0      real   NaN              0.122932  0.000376  36.798453  0.003951   
1   uniform   NaN              0.124060  0.000376  38.085759  0.009677   
2  gaussian   8.0              0.119925  0.000376  34.692228  0.473111   

                                          
          N0    N0 err lambda       Chi2  
0  43.272241  0.194533    0.5   0.008776  
1  52.815583  0.497892    0.5  10.872947  
2  36.599979  0.010746    0.5   0.208704

# Fokker-Planck and Nekhoroshev

In [47]:
import crank_nicolson_numba.nekhoroshev as nk

In [48]:
k = fitting_data[("Model 4 (3 free pars)", "k")][1]
rho = fitting_data[("Model 4 (3 free pars)", "rho")][1]
N0 = fitting_data[("Model 4 (3 free pars)", "N0")][1]

## 4D normal distribution in 1D Action variable
If we consider the following distribution:

$$ f(x,p_x,y,p_y) = \frac{1}{4\pi^2\sigma^4}\exp\left[-\frac{1}{2}\left(\frac{x^2}{\sigma^2}+\frac{p_x^2}{\sigma^2}+\frac{y^2}{\sigma^2}+\frac{p_y^2}{\sigma^2}\right)\right]$$

we convert the distribution to polar variables:

$$f(r, \alpha, \theta_1, \theta_2) = \frac{r^3 \sin(\alpha)\cos(\alpha)}{4\pi^2\sigma^4} \exp\left[-\frac{1}{2}\left(\frac{r^2}{\sigma^2}\right)\right]$$

we then consider distribution integrated over $\alpha, \theta_1, \theta_2$:

$$f(r) = \int_0^{2\pi}\int_0^{2\pi}\int_0^{\pi/2} f(r,\alpha,\theta_1,\theta_2)\,d\alpha\,d\theta_1\,d\theta_2 = \frac{r^3}{2\sigma^4}\exp\left[-\frac{1}{2}\left(\frac{r^2}{\sigma^2}\right)\right]$$


and then we consider $I = r^2/2$ which implies also $dr = dI/I^{1/2}$, and end up with:

$$f(I) = \frac{I}{\sigma^4}\exp\left[-\frac{I}{\sigma^2}\right] = c I \exp\left[-\frac{I}{\sigma^2}\right]$$

where $c$ will be normalized in order to have a unitary starting value for the distributions considered.

## Nekhoroshev-like Diffusion equation

$$D(I) = c \exp\left[-\left(\frac{I_\ast}{I}\right)^{1/2\kappa}\right]$$

$$D(I) = c \exp\left[-2\left(\frac{I_\ast}{I}\right)^{1/2\kappa}\right]$$

(verificare su articolo)

* Try to use free $\kappa$
* (done) Fix $\kappa$ 

### Parameters setup

In [49]:
I_max = DA0**2 / 2
I, stepI = np.linspace(0, I_max, 3000, retstep=True)
I0 = I * np.exp(-(I/sigma**2))

I0 = nk.logistic_damping(I, I0, 470, 10)

I0 /= integrate.trapz(I0, I)

c = 1.0

I_star_min = 150.0
I_star_max = 350.0
I_star_steps = 401
I_star_list = np.linspace(I_star_min, I_star_max, I_star_steps)

iter_step = 100

In [50]:
t1_fig, t1_ax = plt.subplots()
t1_ax.plot(I, I0)
t1_ax.set_xlabel("$I$")
t1_ax.set_ylabel("$f(I)$")
t1_ax.set_title("Starting distribution for FP (logistic dumping added at the end)")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Starting distribution for FP (logistic dumping added at the end)')

In [51]:
best_data, best_I_star, best_time, error = fit.scan_I_star(
    values,
    turn_sampling,
    I_max,
    I0,
    k,
    200.0,
    1.0,
    iter_step,
    0.05
)

Step: 0 / 401 , Number of steps executed 1501 , dt: 1.0
Increasing dt!
Step: 1 / 401 , Number of steps executed 754 , dt: 2.0
Increasing dt!
Step: 2 / 401 , Number of steps executed 379 , dt: 4.0
Step: 3 / 401 , Number of steps executed 381 , dt: 4.0
Step: 4 / 401 , Number of steps executed 383 , dt: 4.0
Step: 5 / 401 , Number of steps executed 384 , dt: 4.0
Step: 6 / 401 , Number of steps executed 386 , dt: 4.0
Step: 7 / 401 , Number of steps executed 388 , dt: 4.0
Step: 8 / 401 , Number of steps executed 390 , dt: 4.0
Step: 9 / 401 , Number of steps executed 391 , dt: 4.0
Step: 10 / 401 , Number of steps executed 393 , dt: 4.0
Step: 11 / 401 , Number of steps executed 395 , dt: 4.0
Step: 12 / 401 , Number of steps executed 397 , dt: 4.0
Step: 13 / 401 , Number of steps executed 399 , dt: 4.0
Step: 14 / 401 , Number of steps executed 401 , dt: 4.0
Step: 15 / 401 , Number of steps executed 403 , dt: 4.0
Step: 16 / 401 , Number of steps executed 405 , dt: 4.0
Step: 17 / 401 , Number of 

In [53]:
print("Displaying the best FP values obtained:")
print("I_star = {}".format(best_I_star))
print("Time scale = {}".format(best_time[0]/axis_sampling[0]))

Displaying the best FP values obtained:
I_star = 259.5
Time scale = 416.768


In [54]:
fp2_fig, fp2_ax = plt.subplots()
fp2_ax.plot(axis_sampling, values, label="Loss from original DA")
fp2_ax.plot(axis_sampling, best_data, label="FP process $(I_\\ast={})$".format(best_I_star))
fp2_ax.legend()
fp2_ax.set_xlabel("$N$ turns")
fp2_ax.set_ylabel("Active beam")
fp2_ax.set_title("Comparison between loss curve from DA\nand best equivalent Fokker-Planck process $(\\kappa={:.3}, I_\\ast={})$".format(k, best_I_star))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Comparison between loss curve from DA\nand best equivalent Fokker-Planck process $(\\kappa=0.123, I_\\ast=259.5)$')

## Test with also $\kappa$ somewhat free

In [89]:
k = fitting_data[("Model 4 (3 free pars)", "k")][1]
rho = fitting_data[("Model 4 (3 free pars)", "rho")][1]
N0 = fitting_data[("Model 4 (3 free pars)", "N0")][1]

In [108]:
I_max = DA0**2 / 2
I, stepI = np.linspace(0, I_max, 3000, retstep=True)
I0 = I * np.exp(-(I/sigma**2))

I0 = nk.logistic_damping(I, I0, 470, 15)

I0 /= integrate.trapz(I0, I)

c = 1.0

I_star_min = 250.0
I_star_max = 450.0
I_star_steps = 101
I_star_list = np.linspace(I_star_min, I_star_max, I_star_steps)

iter_step = 100

dk = 0.1
k_list = np.linspace(k, k + dk, 100)

In [ ]:
fit_values, best_k, best_I_star, best_t, best_error = fit.scan_k(
    values,
    turn_sampling,
    I_max,
    I0,
    k,
    270.0,
    1.0,
    iter_step,
    0.05
)